# Intensity of point processes

In [ ]:
import numpy as np
import pandas as pd
import scipy as sp
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

Functions for visualisation:

In [ ]:
def regular_on_rectangle(grid, random_component, x_lim, y_lim):
    dx = (x_lim[1] - x_lim[0])/grid[0]
    dy = (y_lim[1] - y_lim[0])/grid[1]
    mx = np.linspace(x_lim[0] + dx/2, x_lim[1] - dx/2, grid[0])
    my = np.linspace(y_lim[0] + dy/2, y_lim[1] - dy/2, grid[1])
    px,py=np.meshgrid(mx,my)
    rx = np.random.rand(grid[1],grid[0])* dx - dx/2
    ry = np.random.rand(grid[1],grid[0])* dy - dy/2
    px=px+random_component*rx
    py=py+random_component*ry
    p={'X':px.flatten(), 'Y':py.flatten()}
    points=pd.DataFrame(p)
    return points
def homogeneous_poisson_on_rectangle(intensity, x_lim, y_lim):
    a = (abs(x_lim[1]-x_lim[0]))*(abs(y_lim[1]-y_lim[0]))
    ex = intensity*a
    n = np.random.poisson(lam=ex, size=1)
    nn=n[0]
    x=np.random.rand(nn)*(x_lim[1]-x_lim[0])+x_lim[0]
    y=np.random.rand(nn)*(y_lim[1]-y_lim[0])+y_lim[0]
    p={"X":x.flatten(),"Y":y.flatten()}
    points=pd.DataFrame(p)
    return points
def unhomogeneous_poisson_on_rectangle(intensity_function, x_lim, y_lim):
    srx=(x_lim[0]+x_lim[1])/2
    sry=(y_lim[0]+y_lim[1])/2
    max_fun=sp.optimize.minimize(lambda x: -intensity_function(x), (srx, sry), bounds=((x_lim[0], x_lim[1]),(y_lim[0], y_lim[1])))
    points=homogeneous_poisson_on_rectangle(-max_fun.fun,x_lim,y_lim)
    points_probability=1-intensity_function([points["X"],points["Y"]])/-max_fun.fun
    a=np.zeros(points["X"].size)
    for i in range(points["X"].size):
        a[i]=np.random.choice([1,0],p=[points_probability[i],1-points_probability[i]])
    points2_a=np.where(a==0)
    points=points.loc[points2_a]
    return points

## Analysis of point process' intensity via point count on subregions

In [ ]:
def point_count_on_subregions(points, bins, x_lim, y_lim):
    xlim=x_lim[1]-x_lim[0]
    ylim=y_lim[1]-y_lim[0]
    xforall=xlim/bins[0]
    yforall=ylim/bins[1]
    xpoints=np.linspace(x_lim[0], x_lim[1], bins[0]+1)
    ypoints=np.linspace(y_lim[0], y_lim[1], bins[1]+1)
    
    H, xedges, yedges = np.histogram2d(points["X"], points["Y"], bins=(xpoints, ypoints))
    H = H.T
    return [xpoints,ypoints,H]

In [ ]:
def intensity_on_subregions(points, bins, x_lim, y_lim):
    data=point_count_on_subregions(points, bins, x_lim, y_lim)
    x = data[0]
    y = data[1]
    H = data[2]/((x_lim[1]-x_lim[0])*(y_lim[1]-y_lim[0]))
    return [x,y,H] 

### Generating exemplary process

In [ ]:
x_lim = [-10,10]
y_lim = [-5,5]
r_poisson = regular_on_rectangle([40,20], 0.5, x_lim, y_lim)
h_poisson = homogeneous_poisson_on_rectangle(20, x_lim, y_lim)
nh_poisson = unhomogeneous_poisson_on_rectangle(lambda x: 10*(np.cos(np.pi/4*x[0])+1), x_lim, y_lim)

In [ ]:
regions=[40,20]

rp = intensity_on_subregions(r_poisson, regions, x_lim, y_lim)
hp=intensity_on_subregions(h_poisson, regions, x_lim, y_lim)
nhp=intensity_on_subregions(nh_poisson, regions, x_lim, y_lim)

### Visualisation

In [ ]:
plt.rc('xtick', labelsize=10)
plt.rc('ytick', labelsize=10)
plt.rc('axes', labelsize=15)

plots=[rp,hp,nhp]
plot_num=[131,132,133]
titles=["Regular Poisson", "Homogenous Poisson", "Unhomogenous Poisson"]
axList=[]

fig = plt.figure(figsize=[15,8])
fig.suptitle('Point count on subregions', fontsize=30)

for i in range(3):
    ax = fig.add_subplot(plot_num[i],aspect='equal')
    ax.set_title(titles[i], fontsize=15)
    plt.subplots_adjust(hspace=0.4)
    axx=ax.pcolormesh(plots[i][0], plots[i][1], plots[i][2])
    ax.set(xlabel='X', ylabel='Y')
    axList.append(ax)
    
fig.colorbar(axx,ax=axList, orientation = 'horizontal')

## Analysis of point process' intensity via kernel functions

In [ ]:
def intensity_on_kde(points, kernel_radius, grid, x_lim, y_lim):
    mx=np.linspace(x_lim[0], x_lim[1], grid[0])
    my=np.linspace(y_lim[0], y_lim[1], grid[1])
    px,py=np.meshgrid(mx,my)
    pom=np.zeros(px.size)
    px=px.flatten()
    py=py.flatten()
    p={'X':px, 'Y':py, 'I':pom.flatten()}
    intensity_data=pd.DataFrame(p)
    
    odl_pom_w=np.zeros(points['X'].size)
    
    k=kernel_radius**2
    s=3/np.pi/k
    for i in range(intensity_data['I'].size):
        px0=px[i]-points['X']
        py0=py[i]-points['Y']
        odl_pom_w=px0**2+py0**2
        odl_pom=odl_pom_w[odl_pom_w<k]
        intensity_data['I'][i]=sum(s*(1-odl_pom/k)**2)
    return intensity_data

### Generating exemplary process

In [ ]:
r=1.5
grid=[200,100]
x_lim = [-10,10]
y_lim = [-5,5]

intensity_data_r=intensity_on_kde(r_poisson, r, grid, x_lim, y_lim)
intensity_data_h=intensity_on_kde(h_poisson, r, grid, x_lim, y_lim)
intensity_data_uh=intensity_on_kde(nh_poisson, r, grid, x_lim, y_lim)

### Visualisation

In [ ]:
fig=plt.figure(figsize=(15,8))
fig.suptitle("Badanie intensywności procesu punktowego metodą funkcji jądrowych", fontsize=20)

data=[intensity_data_r, intensity_data_h,intensity_data_uh]
plot_num=[131,132,133]
titles=["Regular Poisson", "Homogenous Poisson", "Unhomogenous Poisson"]
axDataList=[]

for i in range(3):
    ax = fig.add_subplot(plot_num[i])
    ax.set_aspect('equal', 'box')
    a=ax.tricontourf(data[i]["X"],data[i]["Y"],data[i]["I"])
    ax.set_title(titles[i],fontsize=15)
    ax.set(xlabel='X',ylabel='Y')
    axDataList.append(ax)
    
plt.subplots_adjust(hspace=0.4)
fig.colorbar(a,ax=axDataList,orientation = 'horizontal')

## Chi-squared method

In [ ]:
def distribution_table(bin_counts):
    min_ob=bin_counts.min()
    max_ob=bin_counts.max()
    K=np.arange(min_ob,max_ob+1)
    n=[]
    for i in K:
        n.append(np.sum(bin_counts==i))
    return pd.DataFrame({"K":K.flatten(), "N(K)":n})

def poisson_distribution_table(k, mu):
    p=sp.stats.poisson.pmf(k, mu)
    p=p/np.sum(p)
    return pd.DataFrame({"K":k, "P(K)":p})

def pearsons_chi2_test(tested_distribution, theoretical_distribution, alpha, ddof):
    chi_kwadrat=0
    n=0
    s=np.size(tested_distribution["K"])
    n=sum(tested_distribution["N(K)"])
    chi_sq=np.sum((tested_distribution["N(K)"]-n*theoretical_distribution["P(K)"])**2/n/theoretical_distribution["P(K)"])
    chi_sq_alpha=sp.stats.chi2.ppf(1-alpha,s-1)
    
    print("Chi-squared Pearson test")
    print("H0: Tested distribution is consistent with the theoretical distribution")
    print("H1: Tested distribution is not consistent with the theoretical distribution")
    print("chi2 = {} chi2_alfa = {}".format(chi_sq, chi_sq_alpha))
    if(chi_sq<chi_sq_alpha):
        print("chi2 < chi2_alpha")
        print("The test results do not give the basis to reject H0 hypothesis in favor of the H1 on the alpha level = {}".format(alpha))
    else:
        print("chi2 >= chi2_alpha")
        print("Rejection of the H0 hypothesis in favor of H1 on the alpha level = {}".format(alpha))

### Chi-squared test on unhomogonous Poisson process

In [ ]:
count=point_count_on_subregions(nh_poisson, [40,20],[0,20],[0,10])[2].T
dt=distribution_table(count)
dpt=poisson_distribution_table(dt["K"],20*(20*10)/(40*20))
pearsons_chi2_test(dt, dpt, 0.05, 0)

## Kolmogorov–Smirnov test

In [ ]:
def kolmogorow_smirnow_test(tested_points, theoretical_points, alpha, ddof):
    l_a=sp.stats.kstwobign.ppf(1-alpha)
    D, pvalue = sp.stats.kstest(tested_points.squeeze(),theoretical_points.squeeze())
    l = D*tested_points.size**0.5
    print("Kolmogorov–Smirnov test for {} coordinate".format(tested_points.columns[0]))
    print("H0: Tested distribution is consistent with the theoretical distribution")
    print("H1: Tested distribution is not consistent with the theoretical distribution")
    print("lambda = {} lambda_alpha = {}".format(round(l,3), round(l_a,3)))
    if(l<l_a):
        print("lambda < D_alpha")
        print("The test results do not give the basis to reject H0 hypothesis in favor of the H1 on the alpha level = {}".format(alpha))
    else:
        print("lambda >= D_alpha")
        print("Rejection of the H0 hypothesis in favor of H1 on the alpha level = {}".format(alpha))

### Kolmogorov–Smirnov test on unhomogonous Poisson process

In [ ]:
theoretical_points=np.linspace(0,20,np.size(nh_poisson["X"]))
kolmogorow_smirnow_test(nh_poisson[["X"]], theoretical_points, 0.05, 0)
print()
theoretical_points=np.linspace(0,10,np.size(nh_poisson["Y"]))
kolmogorow_smirnow_test(nh_poisson[["Y"]], theoretical_points, 0.05, 0)